In [1]:
!pip install minisom

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from minisom import MiniSom
from sklearn.neighbors import KNeighborsClassifier
import pickle
# Evaluate performance
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score



##urls

In [3]:
# Define the function for normalization using tanh-estimator method, excluding specified columns
def normalize_with_tanh_estimator(data, mean_std_dict, exclude_cols):
    normalized_data = pd.DataFrame()
    for col in data.columns:
        if col not in exclude_cols:
            mu, sigma = mean_std_dict[col]
            normalized_col = 0.5 * (np.tanh(0.1 * ((data[col] - mu) / sigma)) + 1)
            normalized_data[col] = normalized_col
        else:
            normalized_data[col] = data[col]  # Keep excluded columns as they are
    return normalized_data

In [4]:
# Function to calculate distance between input sample V and reference point G
def calculate_distance(v, g):
    return np.linalg.norm(v - g)

In [5]:
# Function to classify input sample as attack or normal and update y_pred array
def classify_and_update(sample, d_threshold, sigma, y_pred, g):
  for sam in sample:
    distance_to_g = calculate_distance(sam, g)
    # print("distance_to_g", distance_to_g)
    p_d_greater_than_x = 1 - np.exp(- (distance_to_g / sigma)**2)  # Cumulative distribution function
    is_attack = distance_to_g > d_threshold or p_d_greater_than_x > 0.6  # You can adjust the threshold as needed
    y_pred.append(1 if is_attack else 0)

  return y_pred


In [6]:
def find_best_values(input_sample, g, testlabels):
  sigma_min = 0.001
  sigma_max = 0.5
  # sigma_step = 0.001
  sigma_step = 0.1
  d_threshold_min = 0.001
  d_threshold_max = 0.5
  # d_threshold_step = 0.001
  d_threshold_step = 0.1
  # Generate arrays of varying values for sigma and d_threshold
  sigma_values = np.arange(sigma_min, sigma_max + sigma_step, sigma_step)
  d_threshold_values = np.arange(d_threshold_min, d_threshold_max + d_threshold_step, d_threshold_step)

  # Initialize an empty dataframe
  df_results = pd.DataFrame(columns=['sigma', 'd_threshold', 'accuracy', 'FDR'])
  max = len(sigma_values)*len(d_threshold_values)
  i=1
  # Initialize an empty list to store results
  results_list = []

  # Example usage loop to calculate accuracy and FDR for each parameter combination
  for sigma in sigma_values:
      for d_threshold in d_threshold_values:
          y_pred = []  # Initialize y_pred for each combination of parameters
          y_pred = classify_and_update(input_sample, d_threshold, sigma, y_pred, g)
          ypred = np.array(y_pred)

          # Evaluate performance
          report = classification_report(testlabels, y_pred, zero_division=1, output_dict=True)
          accuracy = report['accuracy']

          # Calculate False Detection Rate (FDR)
          precision_recall = report['1']  # Assuming '1' represents DDoS class
          FDR = (1 - precision_recall['precision']) * 100

          # Append the results to the list
          results_list.append({'sigma': sigma, 'd_threshold': d_threshold, 'accuracy': accuracy, 'FDR': FDR})
          # print(f"\r{i/max * 100}--- {i} OF {max} accuracy at {accuracy} FDR AT {FDR}")
          i=i+1


  # Assuming df_results contains the data from the previous code snippets
  print(results_list)
  # Convert the list of dictionaries to a DataFrame
  # Create the DataFrame after the loop
  df_results = pd.DataFrame(results_list)
  # Find the row with maximum accuracy
  max_accuracy_row = df_results[df_results['accuracy'] == df_results['accuracy'].max()]
  min_fdr_row = df_results[df_results['FDR'] == df_results['FDR'].min()]
  # Extract the corresponding sigma and d_threshold values
  max_accuracy_sigma = max_accuracy_row['sigma'].values[0]
  max_accuracy_d_threshold = max_accuracy_row['d_threshold'].values[0]
  max_accuracy_accuracy = max_accuracy_row['accuracy'].values[0]
  max_accuracy_fdr = max_accuracy_row['FDR'].values[0]

  min_fdr_sigma = min_fdr_row['sigma'].values[0]
  min_fdr_d_threshold = min_fdr_row['d_threshold'].values[0]
  min_fdr_accuracy = min_fdr_row['accuracy'].values[0]
  min_fdr_fdr = min_fdr_row['FDR'].values[0]

  # Create the DataFrame after the loop
  return {'max_accuracy_sigma': max_accuracy_sigma, 'max_accuracy_d_threshold': max_accuracy_d_threshold, 'max_accuracy_accuracy': max_accuracy_accuracy, 'max_accuracy_fdr': max_accuracy_fdr, 'min_fdr_sigma': min_fdr_sigma, 'min_fdr_d_threshold': min_fdr_d_threshold,'min_fdr_accuracy': min_fdr_accuracy, 'min_fdr_fdr': min_fdr_fdr}


In [7]:
def main_best_match(df, com, bandflag):

  if bandflag == 1:
    # Check if the column exists
    column_to_drop = 'bandwidth'
    if column_to_drop in df.columns:
        # Drop the column
        df.drop(columns=[column_to_drop], inplace=True)
        print(f"\n'{column_to_drop}' column dropped.")


  # Define the columns to exclude from normalization
  exclude_cols = ['label']  # Add more columns here if needed
  # Calculate and store the mean and standard deviation for each dimension, excluding specified columns
  mean_std_dict = {}
  for col in df.columns:
      if col not in exclude_cols:
          mean_std_dict[col] = (df[col].mean(), df[col].std())

  # Normalize the dataset using tanh-estimator method, excluding specified columns
  normalized_df = normalize_with_tanh_estimator(df, mean_std_dict, exclude_cols)


  # Filter rows where label equals 0
  filtered_df = normalized_df[normalized_df['label'] == 0]

  # Concatenate the filtered rows into a new DataFrame
  result_df = pd.concat([filtered_df[col] for col in filtered_df.columns], axis=1)

  # Reset index if needed
  result_df.reset_index(drop=True, inplace=True)

  traindf = (result_df)

  X = normalized_df.iloc[:, :-1].values
  y = normalized_df.iloc[:, -1].values

  trainX = traindf.iloc[:, :-1].values
  trainy = traindf.iloc[:, -1].values

  # Splitting dataset into training and testing
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  # Initialize and train SOM
  som_size = 30  # Example size, adjust based on your dataset

  som = MiniSom(som_size, som_size, X_train.shape[1], sigma=15, learning_rate=0.1, random_seed=0 )

  som.random_weights_init(trainX)

  u_matrixfirst = som.distance_map()  # Get the U-matrix
  # Initialize list to store quantization errors
  quantization_errors = []
  som.train_batch(trainX, 1000, verbose = "true")

  u_matrixlast = som.distance_map()  # Get the U-matrix

  # Plot the U-Matrix before training
  plt.figure(figsize=(12, 6))
  plt.subplot(1, 2, 1)
  plt.title("U-Matrix Before Training")
  plt.pcolor(u_matrixfirst, cmap='viridis')
  plt.colorbar()

  # Plot the U-Matrix after training
  plt.subplot(1, 2, 2)
  plt.title("U-Matrix After Training")
  plt.pcolor(u_matrixlast, cmap='viridis')
  plt.colorbar()

  plt.tight_layout()
  plt.show()


  # testvalues = X_test
  # testvalues = X_train
  testvalues = X
  # testlabels = y_train
  testlabels = y


  # Calculate the reference point G
  g = np.median(som.get_weights(), axis=(0, 1))


  # Example usage
  input_sample = testvalues  # Example input sample (you can replace this with your actual data)
  d_threshold = 0.13309999999999966  # Predefined distance threshold

  sigma =  0.139099999999999  # Probability threshold

  y_pred = []

  y_pred=classify_and_update(input_sample, d_threshold, sigma, y_pred, g)
  ypred = np.array(y_pred)

  # report = classification_report(testlabels, ypred, zero_division=1)
  # print(report)
  # print("here1")
  # print(len(testlabels))
  # print(len(y_pred))
  accuracy = accuracy_score(testlabels, y_pred)
  # print("Accuracy:", accuracy)
  # print(classification_report(testlabels, y_pred, zero_division=1))
  # print("here2")
  # print("Here")
  precision_recall = classification_report(testlabels, y_pred, output_dict=True, zero_division=1)
  precision_ddos = precision_recall['1']  # Assuming '1' represents DDoS class

  FDR = (1 - precision_ddos['precision'])* 100
  print(f"False Detection Rate (FDR): {FDR:.2f}%")

  #ROC Curve

  fpr, tpr, thresholds = roc_curve(testlabels, y_pred)
  roc_auc = auc(fpr, tpr)
  plt.figure(figsize=(8, 6))
  plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
  plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic (ROC) Curve')
  plt.legend(loc='lower right')
  plt.show()

  best_values = find_best_values(input_sample, g, testlabels)

  # print(best_values

  return com, accuracy, FDR, roc_auc, precision_ddos['precision'], sigma, d_threshold, g, mean_std_dict, best_values

In [8]:
# # Load the dataset

# urls = []
# comments = []

# # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/combined%20Dataset.csv")
# # comments.append("1s split")
# # #self made data
# # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/som_data.csv")
# # comments.append("self made data")
# # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/1ssplit%20combined%20dataset%20CAIDA%202007.csv")
# # comments.append("1s split with ")

# #new 2s splits
# # #filled
# # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20filled.csv")
# # comments.append("filled 2s split with band")
# # #unfilled
# # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20unfilled.csv")
# # comments.append("unfilled 2s split with band")

# # #filled
# # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20filled.csv")
# # comments.append("filled 2s split without band")
# #unfilled
# urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20unfilled.csv")
# comments.append("unfilled 2s split without band")

# ### ids 2017 benign only
# ##filled
# urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_filled_entropy.csv")
# comments.append("ids 2017 benign only filled")
# # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_filled_entropy.csv")
# # comments.append("ids 2017 benign only filled without band")
# ###unfilled
# urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_unfilled_entropy.csv")
# comments.append("ids 2017 benign only unfilled")
# # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_unfilled_entropy.csv")
# # comments.append("ids 2017 benign only unfilled without band")


# resultant_columns = ['comments', 'accuracy', 'FDR', 'roc_auc', 'precision', 'sigma', 'd_threshold', 'g', 'mean_std_dict', 'best_values']
# # resultant_df = pd.DataFrame(columns=resultant)
# resultant=[]
# iteration = 0
# for url in urls:
#   com = comments[iteration]
#   if 'without band' in com:
#     bandflag = 1
#   else:
#     bandflag = 0

#   df = pd.read_csv(url)
#   resultant.append(main(df,com, bandflag))
#   iteration += 1

In [9]:
# resultant_df = pd.DataFrame(resultant, columns=resultant_columns)

In [10]:
# print(resultant)

In [11]:
# resultant_df


In [12]:
# for resultant_d in resultant_df['g']:
#   print(resultant_d)

In [13]:
# resultant_df.to_csv('resultant_df.csv')

In [14]:
# resultant_df.to_parquet('resultant_df.parquet')

In [15]:
def main_master_main_learningrate (df, com, bandflag, learning_rate):

  if bandflag == 1:
    # Check if the column exists
    column_to_drop = 'bandwidth'
    if column_to_drop in df.columns:
        # Drop the column
        df.drop(columns=[column_to_drop], inplace=True)
        print(f"\n'{column_to_drop}' column dropped.")


  # Define the columns to exclude from normalization
  exclude_cols = ['label']  # Add more columns here if needed
  # Calculate and store the mean and standard deviation for each dimension, excluding specified columns
  mean_std_dict = {}
  for col in df.columns:
      if col not in exclude_cols:
          mean_std_dict[col] = (df[col].mean(), df[col].std())

  # Normalize the dataset using tanh-estimator method, excluding specified columns
  normalized_df = normalize_with_tanh_estimator(df, mean_std_dict, exclude_cols)


  # Filter rows where label equals 0
  filtered_df = normalized_df[normalized_df['label'] == 0]

  # Concatenate the filtered rows into a new DataFrame
  result_df = pd.concat([filtered_df[col] for col in filtered_df.columns], axis=1)

  # Reset index if needed
  result_df.reset_index(drop=True, inplace=True)

  traindf = (result_df)

  X = normalized_df.iloc[:, :-1].values
  y = normalized_df.iloc[:, -1].values

  trainX = traindf.iloc[:, :-1].values
  trainy = traindf.iloc[:, -1].values

  # Splitting dataset into training and testing
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  # Initialize and train SOM
  som_size = 30  # Example size, adjust based on your dataset

  som = MiniSom(som_size, som_size, X_train.shape[1], sigma=15, learning_rate=learning_rate, random_seed=0 )

  som.random_weights_init(trainX)

  u_matrixfirst = som.distance_map()  # Get the U-matrix
  # Initialize list to store quantization errors
  quantization_errors = []
  som.train_batch(trainX, 1000, verbose = "true")

  u_matrixlast = som.distance_map()  # Get the U-matrix

  # # Plot the U-Matrix before training
  # plt.figure(figsize=(12, 6))
  # plt.subplot(1, 2, 1)
  # plt.title("U-Matrix Before Training")
  # plt.pcolor(u_matrixfirst, cmap='viridis')
  # plt.colorbar()

  # # Plot the U-Matrix after training
  # plt.subplot(1, 2, 2)
  # plt.title("U-Matrix After Training")
  # plt.pcolor(u_matrixlast, cmap='viridis')
  # plt.colorbar()

  # plt.tight_layout()
  # plt.show()


  # testvalues = X_test
  # testvalues = X_train
  testvalues = X
  # testlabels = y_train
  testlabels = y


  # Calculate the reference point G
  g = np.median(som.get_weights(), axis=(0, 1))


  # Example usage
  input_sample = testvalues  # Example input sample (you can replace this with your actual data)
  d_threshold = 0.13309999999999966  # Predefined distance threshold

  sigma =  0.139099999999999  # Probability threshold

  y_pred = []

  y_pred=classify_and_update(input_sample, d_threshold, sigma, y_pred, g)
  ypred = np.array(y_pred)

  # report = classification_report(testlabels, ypred, zero_division=1)
  # print(report)
  # print("here1")
  # print(len(testlabels))
  # print(len(y_pred))
  accuracy = accuracy_score(testlabels, y_pred)
  # print("Accuracy:", accuracy)
  # print(classification_report(testlabels, y_pred, zero_division=1))
  # print("here2")
  # print("Here")
  precision_recall = classification_report(testlabels, y_pred, output_dict=True, zero_division=1)
  precision_ddos = precision_recall['1']  # Assuming '1' represents DDoS class

  FDR = (1 - precision_ddos['precision'])* 100
  print(f"False Detection Rate (FDR): {FDR:.2f}%")

  #ROC Curve

  # fpr, tpr, thresholds = roc_curve(testlabels, y_pred)
  # roc_auc = auc(fpr, tpr)
  # plt.figure(figsize=(8, 6))
  # plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
  # plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
  # plt.xlim([0.0, 1.0])
  # plt.ylim([0.0, 1.05])
  # plt.xlabel('False Positive Rate')
  # plt.ylabel('True Positive Rate')
  # plt.title('Receiver Operating Characteristic (ROC) Curve')
  # plt.legend(loc='lower right')
  # plt.show()

  # best_values = find_best_values(input_sample, g, testlabels)

  return FDR, accuracy,som,g,mean_std_dict

In [16]:
def master_main_learningrate(df, com, bandflag, learning_rate):

  fdr, accuracy,som, g, mean_std_dict = main_create_model_and_g(df, com, bandflag, learning_rate)

  return fdr, accuracy

In [17]:

def master_main_learningrate1():
  # Load the dataset
  urls = []
  comments = []

  # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/combined%20Dataset.csv")
  # comments.append("1s split")
  # #self made data
  # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/som_data.csv")
  # comments.append("self made data")
  # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/1ssplit%20combined%20dataset%20CAIDA%202007.csv")
  # comments.append("1s split with ")

  #new 2s splits
  # #filled
  # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20filled.csv")
  # comments.append("filled 2s split with band")
  # #unfilled
  urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20unfilled.csv")
  comments.append("unfilled 2s split with band")

  # #filled
  # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20filled.csv")
  # comments.append("filled 2s split without band")
  #unfilled
  # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20unfilled.csv")
  # comments.append("unfilled 2s split without band")

  ### ids 2017 benign only
  ##filled
  # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_filled_entropy.csv")
  # comments.append("ids 2017 benign only filled")
  # # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_filled_entropy.csv")
  # # comments.append("ids 2017 benign only filled without band")
  # ###unfilled
  # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_unfilled_entropy.csv")
  # comments.append("ids 2017 benign only unfilled")
  # urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_unfilled_entropy.csv")
  # comments.append("ids 2017 benign only unfilled without band")
  df = pd.read_csv(urls[0])
  com = comments[0]
  bandflag = 0
  result = []
  # from 0.1 to 10 increasing by 0.1
  rangeoflearningrate = np.arange(0.01, 1 + 0.1, 0.01)
  print(rangeoflearningrate)
  for i in rangeoflearningrate :
    fdr, accuracy = master_main_learningrate(df,com, bandflag, i)
    result.append([i ,fdr,accuracy])
  return result


In [18]:
result =main1()

NameError: name 'main1' is not defined

In [ ]:
result

In [ ]:
result_Df = pd.DataFrame(result, columns = ["i ","fdr","accuracy"])

In [ ]:
result_Df